# Household Finance Analysis

In [1]:
import pandas as pd
import re

df = pd.read_csv("household-finances.csv")
df

,date,amount,desc
0,2020-01-04,-650.00,PAYMENT TO DONNICI MP/WOOD CA
1,2020-01-04,-117.84,NRMA PBTM SYDNEY
2,2020-01-06,-42.99,CALLALA CELLARS CALLALA BAY
3,2020-01-06,-50.86,CALLALA BAY HOLDINGS DAVIDSON
4,2020-01-06,-45.55,BAY VILLAGES MEATS CALLALA BAY
...,...,...,...
537,2020-12-10,-19.99,EFTPOS CALLALA CELLARS CALLALA BAY NS
538,2020-12-10,-8.80,EFTPOS BAKEHOUSE DELIGHTS P CALLALA BAY NS
539,2020-12-10,-8.54,EFTPOS CALLALA BAY HOLDING\DAVIDSON AU
540,2020-12-10,2431.28,PAYMENT THANKYOU 823517


In [2]:
df["desc"].value_counts()

EFTPOS CALLALA BAY HOLDING\DAVIDSON  AU          80
CALLALA BAY HOLDINGS  CALLALA BAY                37
EZI*Kemps MA  South Nowra                        35
CALLALA BAY HOLDINGS  DAVIDSON                   33
PAYMENT TO DONNICI MP/WOOD CA                    32
                                                 ..
CULBURRA BAKERY CAKES   Culburra Beac             1
TRANSFER FROM AAI LIMITED T/AS 6001239748         1
LOVATTS PUBLICATNS  GOSFORD                       1
MYKIDSLUNCHBOX  NORTH SYDNEY                      1
TRANSFER FROM MCARE BENEFITS   848027997 AYWQ     1
Name: desc, Length: 167, dtype: int64

In [3]:
df.groupby("desc").sum()

,amount
desc,
ANZ INTERNET BANKING BPAY ORIGIN ENERGY {274447},-465.13
ANZ INTERNET BANKING BPAY ORIGIN ENERGY {817996},-509.99
ANZ INTERNET BANKING BPAY SCC {818203},-1591.32
ANZ INTERNET BANKING BPAY SHOALHAVEN WATER {169196},-313.32
ANZ INTERNET BANKING BPAY SHOALHAVEN WATER {938841},-389.50
...,...
WOOLWORTHS 1436 CULBURRA BEAC,-583.59
WOOLWORTHS 1641 BOMADE BOMADERRY,-54.20
WOOLWORTHS 1648 NOWRA,-101.25


In [4]:
substitutions = {
    "IGA": r".?CALLALA BAY HOLDING.?",
    "WOOLWORTHS": r".?WOOLWORTHS.?",
    "KEMPS": r".?Kemps.?",
    "ENERGY": r".?ORIGIN ENERGY.?",
    "WATER": r".?SHOALHAVEN WATER.?",
    "GAS": r".?TARRANTS GAS.?",
    "PHARMACY": r".?PHARMACY.?",
    "PHARMACY": r".?CHEMIST.?",
    "NOWRA FRESH": r".?NOWRA FARMERS MARKET.?",
    "NOWRA FRESH": r".?NOWRA FRESH.?",
    "TELSTRA": r".?Telstra.?",
    "BOOZE": r".?CELLARS.?"
}

for key, pattern in substitutions.items():
    s = df["desc"].str.replace(pattern, key, flags=re.I)
    df["desc"] = s.combine_first(df["desc"])
    
df["desc"].value_counts()

EFTPOSIGADAVIDSON  AU                                80
IGA  CALLALA BAY                                     37
EZIKEMPSMA  South Nowra                              35
IGA  DAVIDSON                                        33
PAYMENT TO DONNICI MP/WOOD CA                        32
                                                     ..
MYKIDSLUNCHBOX  NORTH SYDNEY                          1
TRANSFER FROM ALLIANZ AUST LTD 000000002004532158     1
CULBURRA BEACH BAMBO  CULBURRA BEAC                   1
LATE PAYMENT FEE                                      1
TRANSFER FROM MCARE BENEFITS   848027997 AYWQ         1
Name: desc, Length: 167, dtype: int64